In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import keras

Using TensorFlow backend.


In [3]:
import albumentations as A

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization,Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D


In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
from ImageDataAugmentor.image_data_augmentor import *

In [8]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose,ToFloat,RandomGamma
)

In [9]:
def augment(aug,image):
    image = aug(image=image)['image']
    return image

def augment_2d(inputs,p=.5):
    return augment(Compose([
        RandomRotate90(p=p),
        Flip(p=p),
        Transpose(p=p),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
        ToFLoat(max_value=255)
    ], p=p),inputs)
    

In [10]:
import cv2
p = 0.5
import albumentations
AUGMENTATIONS_TRAIN = Compose([
   RandomRotate90(p=p),
        Flip(p=p),
        Transpose(p=p),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3)])

AUGMENTATIONS = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
])

AUGMENTATIONS_TEST = Compose([
    ToFloat(max_value=255)
])

In [11]:
import os

In [12]:
train_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=AUGMENTATIONS,
    preprocess_input=None
)
test_datagen = ImageDataAugmentor(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
'images',
target_size=(256,256),
    batch_size=32,
shuffle=True,class_mode='categorical')

Found 194981 images belonging to 39 classes.


In [15]:
def find_id_dir(dirname):
    return [os.path.join(dirname,fname) for fname in sorted(os.listdir(dirname))]

In [16]:
train_generator.image_shape

(256, 256, 3)

def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 20, 5)
        self.conv4 = nn.Conv2d(20, 25, 5)
        
        self.fc1 = nn.Linear(25 * 12*12, 1200)
        self.fc2 = nn.Linear(1200, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, NUM_CLASSES)


In [50]:
def get_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(6,5,input_shape=train_generator.image_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(16,5))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(20,5))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(25,5))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(180))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(85))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dense(39,activation='softmax'))
    return model

In [51]:
def get_opt():
    return tf.keras.optimizers.Adam()


In [52]:
import tensorboard
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

In [53]:
from tensorboard import notebook
notebook.list()

No known TensorBoard instances running.


In [54]:
# stop learining as metric on validatopn stop increasing
early_stopping = EarlyStopping(patience=10, verbose=1, mode = 'auto') 

# tensorboard for monitoring logs
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1,
                          write_graph=True, write_images=True)
callbacks = [tensorboard]


In [ ]:
model = get_model()
opt = get_opt()
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.fit_generator(train_generator,epochs=10)

Epoch 1/10
 126/6094 [..............................] - ETA: 12:49 - loss: 3.8401 - acc: 0.0412

In [ ]:
import cv2

In [ ]:
import PIL

In [ ]:
from cv2 import colorChange

In [ ]:
base = '/home/arturx/ml/mushrooms-classification/'
for dirname in find_id_dir('images'):
    for fname in find_id_dir(dirname):
        fname = base+fname
        try:
            img = cv2.cv2.imread(fname, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            print(fname)